In [119]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [120]:


def create_matchups_with_averages(matchups, games_averages):
    # Merge Team A season averages
    matchups_a = matchups.copy()
    matchups_a = matchups_a.merge(
        games_averages,
        left_on=["posteam_A", "Season_A"],
        right_on=["posteam", "Season"],
        suffixes=("", "_A")
    ).drop(columns=["posteam", "Season"])


    # Merge Team B season averages
    matchups_a = matchups_a.merge(
        games_averages,
        left_on=["posteam_B", "Season_B"],
        right_on=["posteam", "Season"],
        suffixes=("", "_B")
    ).drop(columns=["posteam", "Season"])
    return matchups_a

def create_one_matchup_per_team(alltime_df):
    df = alltime_df.copy().reset_index(drop=True)
    pairs = []
    
    teams = df.to_dict('records')  # list of dicts

    for teamA in teams:
        # randomly select a different team for B
        teamB = random.choice([t for t in teams if t['posteam'] != teamA['posteam']])
        
        pair = {
            "posteam_A": teamA["posteam"],
            "Season_A": teamA["Season"],
            "posteam_B": teamB["posteam"],
            "Season_B": teamB["Season"],
        }
        
        # add stats
        for col in df.columns:
            if col not in ["posteam", "Season"]:
                pair[f"{col}"] = teamA[col]
                pair[f"{col}_B"] = teamB[col]
        
        pairs.append(pair)
    
    return pd.DataFrame(pairs)
def createMatchups(games):
    teamA = games.groupby("GameID").nth(0).reset_index()
    teamB = games.groupby("GameID").nth(1).reset_index()


    matchups = teamA.merge(
        teamB,
        on="GameID",
        suffixes=('_A', '_B')
    )
    return matchups

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

def training(matchups2009a):

    # 1. Define features and target
    drop_cols = [
        'posteam_A','posteam_B','Season_A','Season_B',
        'index_A','index_B','Win_B','Win_A','GameID','Touchdown','Touchdown_B', 'win', 'win_B'
    ]
    X = matchups2009a.drop(columns=drop_cols)
    y = matchups2009a['Win_A']
    train_cols = X.columns.tolist()   # keep column order
    

    # 2. Train/test split BEFORE scaling (correct!)
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )

    # 3. Fit scaler on TRAIN only
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled  = scaler.transform(X_test)

    # 4. Train models
    tree = DecisionTreeClassifier(class_weight='balanced', max_depth=9, random_state=42)
    tree.fit(X_train_scaled, y_train)

    logreg = LogisticRegression(C=1, max_iter=10000, class_weight='balanced')
    logreg.fit(X_train_scaled, y_train)

    svm = SVC(kernel='rbf', probability=True)
    svm.fit(X_train_scaled, y_train)

    mlp = MLPClassifier(hidden_layer_sizes=(32,16), max_iter=1000)
    mlp.fit(X_train_scaled, y_train)

    t = 0
    for i in range(10):
        f = accuracy_score(y_test, mlp.predict(X_test_scaled))
        if f > t:
            t = f

    # 5. Evaluate
    print("Decision Tree accuracy:", accuracy_score(y_test, tree.predict(X_test_scaled)))
    print("LogReg accuracy:", accuracy_score(y_test, logreg.predict(X_test_scaled)))
    print("SVM accuracy:", accuracy_score(y_test, svm.predict(X_test_scaled)))
    print("Best MLP accuracy out of 10 tries:", t)

    models = {'Decision Tree': tree, 'LogReg': logreg, 'SVM': svm, 'MLP': mlp}
    for name, model in models.items():
        y_pred = model.predict(X_test_scaled)
        a_win_rate = np.mean(y_pred)
        b_win_rate = 1 - a_win_rate
        print(f"{name} predicts Team A wins {a_win_rate:.2%}, Team B wins {b_win_rate:.2%}")

    # 6. Choose best model (example selects logistic regression)
    best_model = logreg

    # 7. Return everything needed for prediction
    return best_model, scaler, train_cols

In [ ]:

def predict_future(matchups_future, model, scaler, train_cols):
    
    df_future = matchups_future.copy()

    # 1. Use same training columns & same order
    X_future = df_future[train_cols].copy()
    

    # 2. Use the TRAINED scaler (DO NOT re-fit)
    X_scaled = scaler.transform(X_future)

    # 3. Predict probabilities
    proba = model.predict_proba(X_scaled)

    # Model classes are [0, 1] where 1 = A wins
    col_for_A = list(model.classes_).index(1)
    df_future["proba_A_wins"] = proba[:, col_for_A]

    # 4. Winner prediction
    df_future["winner"] = df_future["proba_A_wins"].apply(
        lambda p: "A" if p >= 0.48 else "B"
    )

    df_future["predicted_winner_team"] = df_future.apply(
        lambda r: r["posteam_A"] if r["winner"] == "A" else r["posteam_B"],
        axis=1
    )

    # 5. Clean output
    return df_future[[
        'posteam_A', 'Season_A',
        'posteam_B', 'Season_B',
        'proba_A_wins', 'winner', 'predicted_winner_team'
    ]]



In [123]:
pd.set_option('display.max_columns', None)
df = pd.read_csv("../NFLPlaybyPlay.csv")


C:\Users\ttank\AppData\Local\Temp\ipykernel_2900\1570469817.py:2: DtypeWarning: Columns (25,51) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../NFLPlaybyPlay.csv")


In [124]:
games_df = (df.groupby(['GameID', 'posteam'], as_index=False)
              .agg({
                  
                  'InterceptionThrown': 'sum',
                  'Fumble': 'sum',
                  'Touchdown': 'sum',
                  'FieldGoalResult': lambda x: (x == 'Good').sum(),
                  'Penalty.Yards': 'sum',
                  'Yards.Gained': 'mean',
                  'PuntResult': lambda x: (x == 'Blocked').sum(),
                  'Sack': 'sum',
                  'PosTeamScore': 'max',
                  'DefTeamScore': 'max',
                  'Season': 'first',
                  'FirstDown': 'sum',
                  'PassAttempt': 'sum',
                  'PassOutcome': lambda x: (x == 'Complete').sum(),
                  
                  
              }))
games_df.rename(columns={'PuntResult': 'PuntBlocked'}, inplace=True)
games_df['CompletionPercentage'] = games_df['PassOutcome'] / games_df['PassAttempt']
yards = (df.groupby(['GameID', 'posteam',], as_index=False)
              .agg({
                  'Yards.Gained': 'sum',
              }))
games_df['Yards.Gained'] = yards['Yards.Gained']
games_df['Win'] = (games_df['PosTeamScore'] > games_df['DefTeamScore']).astype(int)
games = games_df[['GameID', 'posteam','Win', 'Season']]


In [125]:
games_averages = games_df.groupby(['posteam', 'Season'], as_index=False).mean()
games_averages = games_averages.drop(columns=['GameID'])
games_averages = games_averages.rename(columns={'Win': 'win'})
games_averages = games_averages.sort_values(by=['win'], ascending=False)


In [126]:
games_averages.head()

,posteam,Season,InterceptionThrown,Fumble,Touchdown,FieldGoalResult,Penalty.Yards,Yards.Gained,PuntBlocked,Sack,PosTeamScore,DefTeamScore,FirstDown,PassAttempt,PassOutcome,CompletionPercentage,win
90,GB,2011,0.6250,0.6875,3.8125,1.4375,49.7500,475.0000,0.0,2.7500,32.1250,22.3750,22.7500,35.3125,23.1875,0.641801,0.8125
153,NE,2016,0.1250,1.4375,3.2500,1.6875,56.8125,448.3750,0.0,1.7500,27.1250,15.6250,23.9375,36.8750,23.8750,0.642576,0.8125
154,NO,2009,0.8125,1.5625,3.8750,1.3750,48.0000,523.9375,0.0,1.3750,29.5625,21.1875,23.7500,35.3750,24.1250,0.681609,0.8125
147,NE,2010,0.3125,0.4375,3.6250,1.3750,44.3750,456.6250,0.0,1.5625,31.1875,19.9375,22.3750,33.5625,21.5000,0.645250,0.8125
38,CAR,2015,0.6875,0.9375,3.5625,1.9375,51.2500,424.3750,0.0,2.2500,30.3750,18.5625,24.7500,32.9375,19.0625,0.580476,0.8125


In [127]:

season_games = games_df.groupby(['posteam', 'Season'], as_index=False).size()
season_games['total_wins'] = games_df.groupby(['posteam', 'Season'])['Win'].transform('sum')
games_averages2009 = games_averages[games_averages['Season'] == 2009]
BestTeams = games_averages.head(10)
half_avg = games_averages[games_averages['Season'] <= 2011]

In [128]:
matchups = createMatchups(games)
matchups2009 = matchups[matchups['Season_A'] == 2009]
half_matchups = matchups[matchups['Season_A'] <= 2011]

In [129]:
matchups2009a = create_matchups_with_averages(matchups2009, games_averages2009)
matchups2009a.head()
matchupsa = create_matchups_with_averages(matchups, games_averages)

In [130]:
matchupsHalf = create_matchups_with_averages(half_matchups, half_avg)
matchupsHalf.tail(100)

,index_A,GameID,posteam_A,Win_A,Season_A,index_B,posteam_B,Win_B,Season_B,InterceptionThrown,Fumble,Touchdown,FieldGoalResult,Penalty.Yards,Yards.Gained,PuntBlocked,Sack,PosTeamScore,DefTeamScore,FirstDown,PassAttempt,PassOutcome,CompletionPercentage,win,InterceptionThrown_B,Fumble_B,Touchdown_B,FieldGoalResult_B,Penalty.Yards_B,Yards.Gained_B,PuntBlocked_B,Sack_B,PosTeamScore_B,DefTeamScore_B,FirstDown_B,PassAttempt_B,PassOutcome_B,CompletionPercentage_B,win_B
668,1336,2011112009,SEA,1,2011,1337,STL,0,2011,0.8750,1.3125,2.1250,1.6250,59.3125,414.3125,0.0000,2.9375,19.7500,18.8750,22.9375,33.5625,19.0625,0.575172,0.4375,0.6250,1.4375,1.4375,1.3125,59.5000,416.5625,0.0625,3.4375,11.8750,24.3125,24.7500,36.3750,18.5625,0.508349,0.1250
669,1338,2011112010,CHI,1,2011,1339,SD,0,2011,1.3125,1.4375,2.1875,1.7500,47.4375,420.8125,0.0000,3.1250,20.6250,20.5625,23.1250,30.8750,17.0000,0.548855,0.5000,1.2500,1.2500,3.1250,1.7500,47.5000,489.1250,0.0000,2.0000,25.0625,22.8750,24.2500,38.3750,23.4375,0.621210,0.5000
670,1340,2011112011,NYG,0,2011,1341,PHI,1,2011,1.0000,1.1250,3.3125,1.1875,60.3750,494.9375,0.0000,1.7500,23.9375,24.6250,24.8750,38.7500,22.6875,0.581103,0.5625,1.6875,1.5000,2.8750,1.5000,50.4375,474.8125,0.0625,2.1875,23.6250,20.3750,25.6250,35.6250,21.0000,0.592281,0.5000
671,1342,2011112100,KC,0,2011,1343,NE,1,2011,1.1250,0.9375,1.3750,1.5000,51.9375,413.7500,0.0000,2.2500,12.7500,20.8750,24.2500,32.5000,19.3750,0.601332,0.3750,0.8125,0.8125,3.6875,1.7500,48.4375,513.9375,0.0000,2.1250,31.4375,21.0000,26.3750,40.0625,25.6875,0.639820,0.8125
672,1344,2011112400,DET,0,2011,1345,GB,1,2011,1.0000,1.0625,3.6250,1.5000,73.9375,526.9375,0.0000,2.5000,29.1875,24.8125,26.2500,45.3125,28.5000,0.638571,0.5000,0.6250,0.6875,3.8125,1.4375,49.7500,475.0000,0.0000,2.7500,32.1250,22.3750,22.7500,35.3125,23.1875,0.641801,0.8125
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
763,1526,2012010111,CAR,0,2011,1527,NO,1,2011,1.1875,0.8750,3.3750,1.5000,65.0000,491.9375,0.0000,2.3750,25.3125,26.5625,24.8125,34.0000,20.0625,0.596871,0.3750,0.8750,0.5625,4.1875,1.8750,43.7500,576.9375,0.0625,1.6875,34.2500,22.4375,27.3750,44.2500,31.4375,0.711328,0.7500
764,1528,2012010112,SF,1,2011,1529,STL,0,2011,0.3750,1.0000,2.1875,2.8750,57.0000,401.7500,0.0625,2.8750,23.2500,14.0000,21.6250,29.8125,18.0000,0.610676,0.8125,0.6250,1.4375,1.4375,1.3125,59.5000,416.5625,0.0625,3.4375,11.8750,24.3125,24.7500,36.3750,18.5625,0.508349,0.1250
765,1530,2012010113,DEN,0,2011,1531,KC,1,2011,0.8750,1.5625,2.3750,1.2500,53.5000,434.0000,0.0000,2.6875,18.1250,24.0000,25.1875,28.6250,14.1875,0.479618,0.2500,1.1250,0.9375,1.3750,1.5000,51.9375,413.7500,0.0000,2.2500,12.7500,20.8750,24.2500,32.5000,19.3750,0.601332,0.3750
766,1532,2012010114,OAK,0,2011,1533,SD,1,2011,1.4375,1.0000,2.6875,1.8750,59.1875,488.6250,0.0000,1.6250,21.5625,24.4375,25.3125,33.8750,20.1875,0.601082,0.5000,1.2500,1.2500,3.1250,1.7500,47.5000,489.1250,0.0000,2.0000,25.0625,22.8750,24.2500,38.3750,23.4375,0.621210,0.5000


In [142]:
Bestmatchup = create_one_matchup_per_team(BestTeams)
m1, scaler, cols = training(matchupsHalf)
predict_future(Bestmatchup, m1, scaler, cols)

['InterceptionThrown', 'Fumble', 'FieldGoalResult', 'Penalty.Yards', 'Yards.Gained', 'PuntBlocked', 'Sack', 'PosTeamScore', 'DefTeamScore', 'FirstDown', 'PassAttempt', 'PassOutcome', 'CompletionPercentage', 'InterceptionThrown_B', 'Fumble_B', 'FieldGoalResult_B', 'Penalty.Yards_B', 'Yards.Gained_B', 'PuntBlocked_B', 'Sack_B', 'PosTeamScore_B', 'DefTeamScore_B', 'FirstDown_B', 'PassAttempt_B', 'PassOutcome_B', 'CompletionPercentage_B']
Decision Tree accuracy: 0.6103896103896104
LogReg accuracy: 0.6753246753246753
SVM accuracy: 0.6363636363636364
Best MLP accuracy out of 10 tries: 0.5974025974025974
Decision Tree predicts Team A wins 48.70%, Team B wins 51.30%
LogReg predicts Team A wins 50.00%, Team B wins 50.00%
SVM predicts Team A wins 40.91%, Team B wins 59.09%
MLP predicts Team A wins 40.91%, Team B wins 59.09%
Index(['InterceptionThrown', 'Fumble', 'FieldGoalResult', 'Penalty.Yards',
       'Yards.Gained', 'PuntBlocked', 'Sack', 'PosTeamScore', 'DefTeamScore',
       'FirstDown', '

,posteam_A,Season_A,posteam_B,Season_B,proba_A_wins,winner,predicted_winner_team
0,GB,2011,NE,2010,0.117327,B,NE
1,NE,2016,IND,2009,0.519479,A,NE
2,NO,2009,IND,2009,0.444941,B,IND
3,NE,2010,SF,2011,0.539647,A,NE
4,CAR,2015,NE,2016,0.377897,B,NE
5,IND,2009,NO,2009,0.519459,A,IND
6,SF,2011,CAR,2015,0.262398,B,CAR
7,NE,2011,IND,2009,0.443108,B,IND
8,DEN,2012,SF,2011,0.557116,A,DEN
9,ATL,2016,NE,2011,0.092829,B,NE


In [ ]:
print(matchups2009a.describe())
print(Bestmatchup[cols].describe())
